This notebook aims to simulate a data scientist consuming data from a data pipeline, using PySpark. 

In this notebook, I will be executing the following steps:
  - Conduct EDA on data produced from Medallion Data Pipeline 
  - Handling Missing Values 

Thereafter, feature engineering, selection and multicollinearity checks will be handled invidually in their respective notebooks. I have specifically done this, since feature engineering should be targeted for LGD, PD, EAD. Doing feature engineering and producing a master table to use for all 3 models makes our model building tedious, computationally expensive and hard to interpret. 

# 1. Import Libraries

In [6]:
# Import function to start Spark
from init_spark import start_spark
spark = start_spark()

from pyspark.sql.functions import (
    col, when, count, desc, isnan, isnull, lit, length, trim, lower, upper, to_date, concat_ws,  regexp_extract, sum 
)

from pyspark.sql.types import (
    StructType, StructField, StringType, DoubleType, IntegerType, DateType, NumericType
)

25/07/03 00:10:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# 2. EDA: Summary Statistics & Identify Data Issues

In this section, I will be acting as a data scientist (credit risk modeling) pulling data from the Gold Delta Layer of the Medallion Structure. I will be mainly observing summary statistics, spotting and solving issues (e.g. missing values), understanding distribution of features etc. [](url)


In [7]:
df = spark.read.format("delta")\
    .load("../data/gold/medallion_cleaned_lc_data")
    
df.limit(10).toPandas()
    

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,default_status
0,46998570,NaN,3750.0,3750.0,3750.0,36,20.49,140.31,E,E4,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
1,49037600,NaN,9675.0,9675.0,9675.0,36,16.55,342.78,D,D2,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
2,47665387,NaN,6050.0,6050.0,6050.0,36,9.99,195.19,B,B3,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
3,47897317,NaN,15000.0,15000.0,15000.0,36,10.99,491.01,B,B4,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
4,49894160,NaN,4200.0,4200.0,4200.0,36,12.29,140.09,C,C1,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,1
5,50043224,NaN,20000.0,20000.0,20000.0,36,10.99,654.68,B,B4,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
6,47807152,NaN,24000.0,24000.0,24000.0,60,9.99,509.82,B,B3,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
7,48765048,NaN,7875.0,7875.0,7875.0,36,16.55,279.01,D,D2,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,1
8,47665594,NaN,13000.0,13000.0,13000.0,36,7.89,406.72,A,A5,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
9,48674698,NaN,11350.0,11350.0,11350.0,36,16.55,402.13,D,D2,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0


In [8]:
df.summary().toPandas()

25/07/03 00:11:12 WARN DAGScheduler: Broadcasting large task binary with size 1546.6 KiB
Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)
25/07/03 00:11:54 WARN DAGScheduler: Broadcasting large task binary with size 1549.8 KiB


,summary,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,settlement_status,settlement_amount,settlement_percentage,settlement_term,default_status
0,count,2221208,0,2221208,2221208,2221208,2221208,2221208,2221208,2221208,...,6897,8969,8969,2221208,2221208,33178,33178,33178,33178,2221208
1,mean,8.006872337822752E7,None,15031.747758426945,15026.667392247822,15011.56822911982,42.876539252514846,13.051038511513125,445.2941312070338,None,...,440.16573727707737,11443.788268480306,189.3172739435832,None,None,None,5027.660913255774,47.704448731087346,13.157785279402013,0.12023007300531963
2,stddev,4.499471628885557E7,None,9178.038578204314,9176.128108494173,9177.664163082049,10.851276559961274,4.8169210367020465,266.6961572525312,None,...,363.0663387257625,7528.382889632833,197.75385252713247,None,None,None,3685.2846237921744,7.315936494507833,8.231626979758742,0.32523045701605563
3,min,54734,None,500.0,500.0,0.0,36,5.31,4.93,A,...,1.92,55.73,0.01,Cash,N,ACTIVE,44.21,0.2,0.0,0
4,25%,44563406,None,8000.0,8000.0,8000.0,36,9.49,251.59,None,...,170.52,5539.91,39.91,None,None,None,2226.0,45.0,6.0,0
5,50%,84013403,None,12825.0,12800.0,12800.0,36,12.62,377.49,None,...,343.83,9878.72,128.02,None,None,None,4173.0,45.0,14.0,0
6,75%,122261275,None,20000.0,20000.0,20000.0,60,15.88,591.92,None,...,595.65,15775.43,276.98,None,None,None,6876.0,50.0,18.0,0
7,max,145647287,None,40000.0,40000.0,40000.0,60,30.99,1719.83,G,...,2680.89,40306.41,1407.86,DirectPay,Y,COMPLETE,33601.0,521.35,181.0,1


Based on the summary statistics of the Lending Club dataset, several data quality issues become apparent. 
- Some columns are unusable, due to to having a **large percentage of missing values** (taken reference from `../sandbox/string_issues`)

- A number of features exhibit **missing values**, including `emp_length` etc. This requires imputation strategies to be implemented

- Some columns are irrelevant to our credit risk modeling project, due to **high cardinality** (large number of unique values), e.g. `emp_title` **(categorical data)**. 

- There are also **redundant columns**, like `member_id`, which provides no value to our prediction of LGD, EAD and PD.

- There are also **post-loan information**. This means that the value of these features are generated after loan origination (attaining application and approval). Hence, such features should be dropped, since they would skew our subsequent machine learning models. Such features include `total_pymnt`, `last_pymnt_d`. These shall be removed in individual modelling notebooks, since models like LGD and EAD will require them. 

- There are features like `delinq_2yrs` which have **outliers** (maximum data point way above the 75% quartile)


# 3. Handling Missing Values

Based on the above issues identified by me with the Lending Club Dataset, I will now be tackling each of them in order. 

### 3.1 Find Null Value % Per Column 
For this credit risk modeling project, I will be dropping columns with &gt; 50% missingness.  Many credit risk modelling projects on Kaggle and Github use 50%-65% missingness as the threshold to drop columns as well. 

However, let's display the columns which have >=50% missing values first to inspect them 

In [9]:

# Get total number of rows
total_rows = df.count()

# Calculate % of nulls per column and keep only those ≥ 50%
missing_val_threshold = 30

high_missingness_columns = []

for column in df.columns:
    null_count = df.select(sum(col(column).isNull().cast("int"))).collect()[0][0]
    null_pct = (null_count / total_rows) * 100
    if null_pct >= missing_val_threshold:
        print(f"{column}: {null_pct:.2f}% null")
        high_missingness_columns.append(column)

# Drop columns with >= 50% missing values (Low predictive power upon inspection)
df = df.drop(*high_missingness_columns) 
print("\n✅ Columns with high pct of missing values dropped ... \n")

# Inspect Dimensions

num_rows = df.count()

num_cols = len(df.columns)

print(f"Updated Shape: ({num_rows}, {num_cols})")

member_id: 100.00% null
loan_status: 100.00% null
mths_since_last_delinq: 51.28% null
mths_since_last_record: 84.11% null
next_pymnt_d: 60.29% null
mths_since_last_major_derog: 74.31% null
annual_inc_joint: 94.79% null
dti_joint: 94.79% null
verification_status_joint: 95.00% null
open_acc_6m: 38.60% null
open_act_il: 38.60% null
open_il_12m: 38.60% null
open_il_24m: 38.60% null
mths_since_rcnt_il: 40.51% null
total_bal_il: 38.60% null
il_util: 47.51% null
open_rv_12m: 38.60% null
open_rv_24m: 38.60% null
max_bal_bc: 38.60% null
all_util: 38.60% null
inq_fi: 38.60% null
total_cu_tl: 38.60% null
inq_last_12m: 38.60% null
mths_since_recent_bc_dlq: 77.01% null
mths_since_recent_revol_delinq: 67.23% null
revol_bal_joint: 95.34% null
sec_app_fico_range_low: 95.34% null
sec_app_fico_range_high: 95.34% null
sec_app_earliest_cr_line: 95.34% null
sec_app_inq_last_6mths: 95.34% null
sec_app_mort_acc: 95.34% null
sec_app_open_acc: 95.34% null
sec_app_revol_util: 95.42% null
sec_app_open_act_il: 95

### 3.2 Dropping Irrelevant/Redundant Columns 
This section implements the removal of **meaningless columns, features which has high cardinality (categorical data), features with little predictive value, e.g. `member_id`, `emp_title` etc, and post-loan features**. Including such features may lead to multicollinearity, and ultimately lead to low predictive power of our credit models. 

Reasons why I removed certain columns are as shown: 
- Columns with `inv`: Largely same as its subset, e.g. `total_pymnt_inv` is largely the same as `total_pymnt`

- `last_pymnt_d` and `last_credit_pull_d` (according to Data Dictionary) have little predictive value even after feature engineering. It merely shows the last payment date by borrower and last date where credit report is pulled. This has little value in predicting PD, LGD or EAD. 

- `sub_grade` is more granular than `grade`. This may lead to a risk of overfitting of our PD, LGD, and EAD models. 

- High Cardinality Columns may lead to high computational costs in encoding for machine learning models, which makes it undesirable in a big data space such as credit risk. 

- Hardship & Settlement Features (Borrowers are only eligible for hardship and settlement programmes after loan origination for Lending Club, not when they apply for it). Borrowers will contact lenders of financial hardship, attempting to settle with lenders for interest-fee payments or lower principal sum payments. Such features should not be used to predict PD, LGD, and EAD. My models should not know if a borrower will fall into hardship for this credit risk modeling project 

- `disbursement_method` indicates how loan funds are delivered to the borrower. This has little relevance in predicting PD, LGD or EAD. 

- 🚩 Low Variance Features may lead to slower running of PCA (which aims to reduce dimensionality). They also add little value to prediction of PD, EAD and LGD. (Dealt after standardisation)


In [10]:
# Drop Derived/Meaningless Features 
derived_features = ["funded_amnt_inv", "sub_grade", "out_prncp_inv", "total_pymnt_inv", "last_pymnt_d", "last_credit_pull_d"] 
df = df.drop(*derived_features)
print(f"✅ Derived/Meaningless Features Dropped ...")


✅ Derived/Meaningless Features Dropped ...


In [11]:
# Drop High Cardinality Features 

# 1. Define Threshold 
high_cardinality_threshold = 50

# 2. Find Categorical Features (to identify high cardinality columns)


categorical_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, StringType)]
print(categorical_cols)

# 3. Identify high-cardinality columns
high_card_cols = []

for col_name in categorical_cols:
    unique_count = df.select(col_name).distinct().count()

    if unique_count >= high_cardinality_threshold:
        print(f"\n{col_name} has {unique_count} unique values → dropping ... ")
        high_card_cols.append(col_name)

# 4. Drop high cardinality columns
df = df.drop(*high_card_cols)

print(f"\n✅ High Cardinality Features Dropped ...")

['grade', 'emp_title', 'home_ownership', 'verification_status', 'pymnt_plan', 'addr_state', 'initial_list_status', 'application_type', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag']

emp_title has 477269 unique values → dropping ... 

addr_state has 51 unique values → dropping ... 

✅ High Cardinality Features Dropped ...


In [12]:
# Drop hardship related columns & miscelleanous columns 
hardship_columns = ["hardship_flag", "disbursement_method", "debt_settlement_flag", 'policy_code']

df = df.drop(*hardship_columns)
print("✅ Hardship & miscelleneous columns dropped ...")

✅ Hardship & miscelleneous columns dropped ...


🚩 I will need to remove post-loan origination features later on for PD prediction. Post-loan origination features such as `recoveries` are needed for LGD, EAD prediction, but not for PD prediction. 

### 3.4 Impute Missing Values (Categorical & Numerical)
After removing unnecessary columns with little predictive power, we will proceed to impute missing values. We will first identify % missing values per column. 

For numerical columns, median values shall replace missing values, given how we haven't dealt with outliers yet. For categorical columns, mode categories shall be used to replace missing values. Such an approach is common and simplistic, though there are advanced imputation techniques like clustering. However, we shall not lose focus of learning about the credit risk modeling domain in this project. 

In [13]:
total_rows = df.count()

for column in df.columns: 
    null_count = df.filter(col(column).isNull()).count()
    if null_count > 0:
        print(f"{column}: {null_count} null values, {round(null_count/total_rows * 100,2)}% missing values.")

emp_length: 142472 null values, 6.41% missing values.
inq_last_6mths: 1 null values, 0.0% missing values.
collections_12_mths_ex_med: 54 null values, 0.0% missing values.
tot_coll_amt: 67219 null values, 3.03% missing values.
tot_cur_bal: 67219 null values, 3.03% missing values.
total_rev_hi_lim: 67219 null values, 3.03% missing values.
acc_open_past_24mths: 46992 null values, 2.12% missing values.
avg_cur_bal: 67219 null values, 3.03% missing values.
bc_open_to_buy: 69761 null values, 3.14% missing values.
bc_util: 70649 null values, 3.18% missing values.
chargeoff_within_12_mths: 54 null values, 0.0% missing values.
mo_sin_old_il_acct: 134567 null values, 6.06% missing values.
mo_sin_old_rev_tl_op: 67219 null values, 3.03% missing values.
mo_sin_rcnt_rev_tl_op: 67219 null values, 3.03% missing values.
mo_sin_rcnt_tl: 67219 null values, 3.03% missing values.
mort_acc: 46992 null values, 2.12% missing values.
mths_since_recent_bc: 68403 null values, 3.08% missing values.
mths_since_rec

In [14]:
# Loop over each column
for feature in df.schema.fields:
    col_name = feature.name
    dtype = feature.dataType

    if isinstance(dtype, StringType):
        mode_value = (
            df.groupBy( col(f"{col_name}")  )
            .count()
            .orderBy(col("count").desc()) 
            .first()[0]
        )

        df = df.fillna({f"{col_name}": mode_value})



    # Impute Numerical Columns with Median
    elif isinstance(dtype, IntegerType) or isinstance(dtype, DoubleType):
        if df.filter(   col(col_name).isNull()  ).count() > 0:
            median_val = df.approxQuantile(col_name, [0.5], 0.01)[0]
            df = df.fillna({col_name: median_val})

print('✅ Categorical Column Missing Values Filled!')
print('✅ Numerical Column Missing Values Filled!')

✅ Categorical Column Missing Values Filled!
✅ Numerical Column Missing Values Filled!


In [15]:
# Double check if there are any missing values before subsequent steps
total_rows = df.count()

output_arr = []
for column in df.columns: 
    null_count = df.filter(col(column).isNull()).count()
    if null_count > 0:
        output_arr.append(f"{column}: {null_count} null values, {round(null_count/total_rows * 100,2)}% missing values.")

if len(output_arr) == 0: 
    print('✅ No Missing Values Found!')
else:
    print(output_arr)

✅ No Missing Values Found!


In [16]:
df.limit(10).toPandas()

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,default_status
0,46998570,3750.0,3750.0,36,20.49,140.31,E,6,RENT,9479.0,...,2.0,100.0,0.0,0.0,0.0,27021.0,11328.0,8700.0,8021.0,0
1,49037600,9675.0,9675.0,36,16.55,342.78,D,6,OWN,45000.0,...,1.0,100.0,50.0,0.0,1.0,52352.0,44743.0,2500.0,19652.0,0
2,47665387,6050.0,6050.0,36,9.99,195.19,B,6,RENT,15180.0,...,0.0,88.9,50.0,0.0,0.0,13700.0,3869.0,5400.0,0.0,0
3,47897317,15000.0,15000.0,36,10.99,491.01,B,6,OWN,70000.0,...,0.0,62.5,0.0,0.0,0.0,55483.0,2796.0,4000.0,0.0,0
4,49894160,4200.0,4200.0,36,12.29,140.09,C,6,MORTGAGE,50000.0,...,6.0,90.0,0.0,2.0,0.0,10800.0,3012.0,7700.0,0.0,1
5,50043224,20000.0,20000.0,36,10.99,654.68,B,6,MORTGAGE,62628.0,...,2.0,90.5,100.0,2.0,0.0,360941.0,34740.0,1800.0,35087.0,0
6,47807152,24000.0,24000.0,60,9.99,509.82,B,6,MORTGAGE,104000.0,...,3.0,96.7,0.0,0.0,0.0,253157.0,22352.0,72000.0,0.0,0
7,48765048,7875.0,7875.0,36,16.55,279.01,D,6,OWN,21900.0,...,3.0,96.0,0.0,0.0,0.0,107315.0,29334.0,53500.0,34946.0,1
8,47665594,13000.0,13000.0,36,7.89,406.72,A,6,MORTGAGE,60000.0,...,2.0,96.9,40.0,0.0,0.0,329636.0,54321.0,16600.0,58836.0,0
9,48674698,11350.0,11350.0,36,16.55,402.13,D,6,OWN,33000.0,...,1.0,86.7,0.0,0.0,0.0,49113.0,29599.0,23800.0,20813.0,0


In [17]:
# df3 represents the DataFrame after finishing Chapter 3 Handling Missing Values 

df.write.format("delta").mode("overwrite").option('mergeSchema', 'true').save("../data/gold/medallion_cleaned_lc_data_b4_model")


In [18]:
# Check if Gold Delta is accessible for subsequent model building 
gold_table = spark.read.format("delta")\
    .load("../data/gold/medallion_cleaned_lc_data_b4_model")
    
gold_table.limit(10).toPandas()

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,default_status
0,69532609,15000.0,15000.0,36,9.80,482.61,B,6,OWN,65000.00,...,6.0,93.3,75.0,0.0,0.0,93750.0,46567.0,19000.0,44150.0,1
1,68721741,19200.0,19200.0,36,9.80,617.73,B,6,OWN,62500.00,...,0.0,100.0,27.3,0.0,0.0,60979.0,16221.0,22200.0,38779.0,0
2,69645747,13800.0,13800.0,36,12.88,464.18,C,6,OWN,30000.00,...,1.0,100.0,20.0,0.0,0.0,32800.0,12892.0,32800.0,0.0,0
3,69979570,8000.0,8000.0,36,10.78,261.08,B,6,RENT,29400.00,...,5.0,82.4,66.7,0.0,0.0,33900.0,14270.0,7300.0,0.0,0
4,69473439,25000.0,25000.0,36,5.32,752.87,A,6,MORTGAGE,180000.00,...,1.0,96.2,20.0,0.0,0.0,708560.0,18418.0,57500.0,21060.0,0
5,70090118,25000.0,25000.0,60,17.27,624.95,D,6,RENT,83471.68,...,2.0,100.0,87.5,0.0,0.0,69311.0,55658.0,21600.0,18192.0,1
6,69654235,19950.0,19950.0,36,16.59,707.21,D,6,RENT,37000.00,...,2.0,100.0,0.0,0.0,0.0,74700.0,29399.0,18100.0,24000.0,0
7,69635724,25000.0,25000.0,60,18.99,648.38,E,6,MORTGAGE,40992.00,...,1.0,93.3,100.0,0.0,1.0,185605.0,12535.0,7700.0,11205.0,0
8,69493668,15000.0,15000.0,36,7.91,469.43,A,6,OWN,51500.00,...,2.0,100.0,66.7,0.0,0.0,38990.0,26787.0,14100.0,12090.0,0
9,70010559,10000.0,10000.0,36,6.99,308.73,A,6,MORTGAGE,36000.00,...,1.0,94.6,0.0,0.0,0.0,181967.0,27119.0,46600.0,18362.0,0


In [19]:
print("✅ All missing values filled and saved to Gold Delta Table!")
print("✅ Ready for PD, LGD, and EAD Modeling!")


✅ All missing values filled and saved to Gold Delta Table!
✅ Ready for PD, LGD, and EAD Modeling!


# 6. WOE & IV Feature Selection 

# 6. Multicollinearity Handling 

# 7. Low Variance Filter

# 8. Save as Master Features 


